In [1]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod_torch import DeepMoD
from deepymod_torch.model.func_approx import NN, Siren
from deepymod_torch.model.library import Library1D
from deepymod_torch.model.constraint import LeastSquares
from deepymod_torch.model.sparse_estimators import Clustering, Threshold
from deepymod_torch.training import train
from deepymod_torch.training.sparsity_scheduler import Periodic
from phimal_utilities.data import Dataset
from phimal_utilities.data.burgers import BurgersDelta
from math import pi

import matplotlib.pyplot as plt

# Setting cuda
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

%load_ext autoreload
%autoreload 2

In [2]:
# Making data
v = 0.1
A = 1.0
x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X_train, y_train = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=2000, noise=0.4, normalize=True)

In [3]:
X_train

tensor([[ 1.0000,  0.6970],
        [ 0.9592, -0.8182],
        [ 0.7959, -0.6768],
        ...,
        [ 0.5918, -0.6162],
        [-0.5102,  0.0505],
        [ 0.1837,  0.0303]])

Note that they are normalized:

In [4]:
X_train.max(dim=0)

torch.return_types.max(
values=tensor([1., 1.]),
indices=tensor([1994, 1884]))

In [5]:
X_train.min(dim=0)

torch.return_types.min(
values=tensor([-1., -1.]),
indices=tensor([1893, 1964]))

# Normal NN baseline

In [6]:
# Configuring model
network = NN(2, [30, 30, 30, 30, 30], 1)  # Function approximator
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint) # Putting it all in the model

# Running model
sparsity_scheduler = Periodic(initial_epoch=1000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True) # Defining optimizer

In [7]:
train(model, X_train, y_train, optimizer, sparsity_scheduler, delta=0.01, patience=200) # Running

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       1000     10.00%             507s   -6.71e+00   5.51e-03   9.68e-05   3.45e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
         True, False])]
       1100     11.00%             495s   -6.69e+00   5.51e-03   2.11e-04   2.13e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
         True, False])]
       1200     12.00%             485s   -6.71e+00   5.49e-03   1.53e-04   2.01e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
        False, False])]
       1300     13.00%             475s   -6.71e+00   5.50e-03   1.35e-04   1.62e+00 [tensor([False,  True,  True, False, False,  True, False, False,  True, False,
        False, False])]
       1400     14.00%             477s   -6.72e+00   5.46e-03   1.29e-04   1.44e+00 [tensor([False,  True,  True, False, False,  True, False, False, False, Fal

In [8]:
# Sparsity masks access to see which terms are active
model.sparsity_masks

[tensor([False, False,  True, False, False,  True, False, False, False, False,
         False, False])]

In [9]:
# These are the coefficients found by the sparsity estimator, i.e. scaled and biased by L1 (often)
model.estimator_coeffs()

[array([[ 0.        ],
        [ 0.        ],
        [ 0.51967191],
        [ 0.        ],
        [ 0.        ],
        [-0.90591649],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]])]

In [10]:
# Constraint coefficients as in the constraint
model.constraint_coeffs(scaled=False, sparse=False)

[tensor([[ 0.0171],
         [-0.6585]], grad_fn=<MmBackward>)]

In [11]:
# For normalized coefficients, set scaled to True
model.constraint_coeffs(scaled=True, sparse=False)

[tensor([[ 0.5268],
         [-0.9255]], grad_fn=<DivBackward0>)]

In [12]:
# If we want the full vector (i.e. with the zeros), set sparse to true
model.constraint_coeffs(scaled=True, sparse=True)

[tensor([[ 0.0000],
         [ 0.0000],
         [ 0.5268],
         [ 0.0000],
         [ 0.0000],
         [-0.9255],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]], grad_fn=<MaskedScatterBackward>)]

# SIREN 

Siren model has the same API as our normal NN:

In [24]:
# Configuring model
network = Siren(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint) # Putting it all in the model

# Running model
sparsity_scheduler = Periodic(initial_epoch=100, periodicity=10) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True) # Defining optimizer

In [25]:
train(model, X_train, y_train, optimizer, sparsity_scheduler, log_dir='runs/SIREN/', delta=0.01, patience=200) # Running

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
        100      1.00%             615s   -6.32e+00   6.06e-03   1.28e-03   2.99e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True,  True,
        False, False])]
[tensor([False,  True,  True, False,  True,  True, False, False,  True,  True,
        False, False])]
[tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
        False, False])]
        125      1.25%             630s   -6.49e+00   5.77e-03   8.37e-04   2.39e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
        False, False])]
[tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
        False, False])]
        150      1.50%             636s   -6.59e+00   5.63e-03   5.40e-04   2.19e+00 [tensor([False,  True,  True, False,  True,  True, False, False,  True, False,
        False, False])]
[tensor([False,  True,  True, False,  T

In [26]:
# Sparsity masks access to see which terms are active
model.sparsity_masks

[tensor([False, False,  True, False, False,  True, False, False, False, False,
         False, False])]

In [27]:
# These are the coefficients found by the sparsity estimator, i.e. scaled and biased by L1 (often)
model.estimator_coeffs()

[array([[ 0.        ],
        [ 0.        ],
        [ 0.56298902],
        [ 0.        ],
        [ 0.        ],
        [-0.88654473],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]])]

In [28]:
# Constraint coefficients as in the constraint
model.constraint_coeffs(scaled=False, sparse=False)

[tensor([[ 0.0176],
         [-0.6526]], grad_fn=<MmBackward>)]

In [29]:
# For normalized coefficients, set scaled to True
model.constraint_coeffs(scaled=True, sparse=False)

[tensor([[ 0.5698],
         [-0.8896]], grad_fn=<DivBackward0>)]

In [30]:
# If we want the full vector (i.e. with the zeros), set sparse to true
model.constraint_coeffs(scaled=True, sparse=True)

[tensor([[ 0.0000],
         [ 0.0000],
         [ 0.5698],
         [ 0.0000],
         [ 0.0000],
         [-0.8896],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]], grad_fn=<MaskedScatterBackward>)]

So SIREN didn't work....